# Dataclass

Na versão 3.7 do Python foi introduzido o módulo `dataclasses`, para simplificar a criação de classes cujo único objetivo é carregar dados relacionados. Esse módulo define o decorador `dataclass`, que gera automaticamente alguns métodos para classes simples.

## Exemplo inicial

Como um exemplo, suponhamos que queremos definir uma classe para representar um ponto num plano bidimensional. Podemos definir uma classe simples da seguinte forma:

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

Note que estou considerando os membros `x` e `y` como públicos. Esse é um ponto importante. Agora podemos usar essa classe para operações simples.

In [ ]:
p1 = Point(10, 20)
origin = Point(0, 0)
print(f'p1 fica em ({p1.x}, {p1.y})')
print(f'A origem fica em ({origin.x}, {origin.y})')

Infelizmente, os objetos dessa classe ainda são um tanto restritos. Por exemplo, não funciona a impressao:

In [ ]:
print(p1)

E também não podemos fazer comparação:

In [ ]:
p2 = Point(10, 20)
p1 == p2

(No código acima, ele simplesmente comparou se os dois objetos são o mesmo.)

Podemos usar o decorador `dataclass` tanto simplificar a definição da classe como definir automaticamente alguns métodos:

In [ ]:
from dataclasses import dataclass

@dataclass
class Point:
    x : float
    y : float

Essa definição diz que os objetos da classe `Point` terão dois campos **públicos**, denominados `x` e `y`, que guardarão valores `float`.

In [ ]:
p1 = Point(10, 20)
origin = Point(0, 0)
print(f'p1 fica em ({p1.x}, {p1.y})')
print(f'A origem fica em ({origin.x}, {origin.y})')

Note como o método `__init__` foi automaticamente definido.

Mas também os métodos `__repr__` e `__eq__` são definidos automaticamente:

In [ ]:
print(p1)

In [ ]:
p2 = Point(10, 20)
p1 == p2

O método `__repr__` é definido de modo a ficar com o formato como o do exemplo acima. O método `__eq__` é definido para comparar um por um os valores dos campos. Neste caso, seria algo como:

```python
class Point:
    ... etc
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
```

## Valores default

Nós podemos também especificar valores default para os campos, que serão usados como valores default no método `__init__` gerado automaticamente:

In [ ]:
@dataclass
class Point:
    x : float = 0
    y : float = 0

In [ ]:
origin = Point()
print(origin)

## Métodos adicionais

Nada impede o programador incluir na classe outros métodos, além dos que vão ser gerados automaticamente. Por exemplo, podemos criar um método para gerar um ponto com certo deslocamento em relação ao nosso ponto e também definir uma forma diferente de converter para cadeias de caracteres:

In [ ]:
@dataclass
class Point:
    x : float = 0
    y : float = 0

    def __str__(self):
        return f'({self.x}, {self.y})'

    def move(self, δx, δy):
        return Point(self.x + δx, self.y + δy)

In [ ]:
p3 = Point(5, 7)
print(p3.move(1, 4))

In [ ]:
p3

## Definição manual de métodos auto-gerados

Às vezes, os métodos gerados automaticamente por `dataclass` não são apropriados. Felizmente, é possível alterar: se o programador define manualmente um dos métodos, esse método será usado e não será gerado automaticamente.

Por exemplo, se gostamos da representação de pontos como a definida no método `__str__` acima e queremos usá-la também para `__repr__`, basta definir o `__repr__`:

In [ ]:
@dataclass
class Point:
    x : float = 0
    y : float = 0

    def __repr__(self):
        return f'({self.x}, {self.y})'

    def move(self, δx, δy):
        return Point(self.x + δx, self.y + δy)

In [ ]:
p3 = Point(5, 7)
print(p3.move(1, 4))

In [ ]:
p3

## Imutabilidade

Os objetos gerados como nos exemplos acima são mutáveis, isto é, podemos acessar os campos e mudar seus valores quando quisermos. 

In [ ]:
p3.y = -10
p3

Isso pode não ser uma boa ideia, pois pode levar a quebra de invariantes. No caso de pontos, isso não é importante, mas se quisermos que os pontos gerados sejam imutáveis (seus valores não poderão ser alterados depois de criados) podemos especificar isso pelo parâmetro `frozen` de `dataclass`:

In [ ]:
@dataclass(frozen=True)
class Point:
    x : float = 0
    y : float = 0

    def __repr__(self):
        return f'({self.x}, {self.y})'

    def move(self, δx, δy):
        return Point(self.x + δx, self.y + δy)

In [ ]:
p4 = Point(5, 7)
p4

In [ ]:
p4.y = -10

## Comparação de ordem

Um outro parâmetro para `dataclass` é o `order`, que especifica que os métodos de comparação por ordem devem ser criados. Neste caso ele faz a comparação lexicograficamente usando a ordem em que os campos foram definidos:

In [ ]:
@dataclass
class Name:
    first : str
    last : str

In [ ]:
js = Name('José', 'Silva')
ms = Name('Maria', 'Silva')
jo = Name('José', 'Oliveira')
print(js <= ms)
print(js >= jo)

In [ ]:
@dataclass(order=True)
class Name:
    first : str
    last : str

In [ ]:
js = Name('José', 'Silva')
ms = Name('Maria', 'Silva')
jo = Name('José', 'Oliveira')
print(js <= ms)
print(js > jo)

## Outros parâmetros de `dataclass`

Entre outros parâmetros para `dataclass` podemos inibir a geração automatica. Por exemplo, se usamos
`repr=False` não será criado automaticamente o método `__repr__`. Similarmente para `init=False` e `eq=False`.

In [ ]:
@dataclass(eq=False)
class NoComp:
    a : int
    b : int

In [ ]:
x = NoComp(1, 2)
y = NoComp(1, 2)
print(x, y)
x == y

## Herança

Fora o fato de que existe geração automatica de métodos, uma `dataclass` é uma classe como qualquer outra. Portanto, podemos usá-las em herança:

In [ ]:
@dataclass(frozen=True)
class PointMass(Point):
    mass : float = 0

In [ ]:
mass1 = PointMass(3.4, 7.2, 0.5)
print(mass1)